# ORA
the general and specfic for KEGG, REACTOME and GO

In [1]:
import pandas as pd
from pypathway import Reactome, GO, KEGG, ORA
from pypathway import ColorectalCancer, IdMapping, GMTUtils
from pypathway import EnrichmentExport
import os
import sys

In [3]:
# load a gmt file.
gmt = GMTUtils.parse_gmt_file("../../tests/assets/gmt_file/h.all.v6.0.entrez.gmt")

In [4]:
# load the example
c = ColorectalCancer()

In [5]:
# infomation of datasets
len(c.deg_list), len(c.background)

(5320, 17216)

In [6]:
res_h = ORA.run(c.deg_list, c.background, gmt)

In [7]:
res_h.table.head()

,name,mapped,number in study,p-value,fdr
0,HALLMARK_APICAL_JUNCTION,185,97,7.750142e-10,3.748583e-09
1,HALLMARK_P53_PATHWAY,181,76,9.962456e-04,1.779010e-03
2,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,186,123,3.256061e-23,5.426768e-22
3,HALLMARK_IL2_STAT5_SIGNALING,188,84,4.488864e-05,1.068777e-04
4,HALLMARK_APICAL_SURFACE,42,19,3.534302e-02,5.355003e-02


In [8]:
# inline bar plot.
res_h.plot()

## KEGG

In [9]:
# kegg enrichment for certain organism
r_kg = KEGG.run(c.deg_list, c.background, 'hsa')

In [10]:
r_kg.table.head()

,ID,Name,mapped,deg,p-value,fdr
0,hsa04972,Pancreatic secretion - Homo sapiens (human),81,33,0.038154,0.106398
1,hsa00590,Arachidonic acid metabolism - Homo sapiens (hu...,49,20,0.090692,0.188991
2,hsa03015,mRNA surveillance pathway - Homo sapiens (human),74,20,0.800729,0.904320
3,hsa04915,Estrogen signaling pathway - Homo sapiens (human),83,35,0.019395,0.063924
4,hsa00514,Other types of O-glycan biosynthesis - Homo sa...,20,3,0.970549,1.000000


In [11]:
r_kg.plot()

## Reactome

In [12]:
# the Example of using the warpper of Reactome gene set enrichment analysis

In [12]:
sybs = [x[1][0] for x in IdMapping.convert(input_id=c.deg_list, species='hsa', source='ENTREZID', target='SYMBOL') if x[1]]

Database org.Hs.eg.db not found, will be downloaded from bioconductor


In [12]:
sybs[:10]

['A2M',
 'MKKS',
 'S100A3',
 'ANKRD29',
 'TMEM250',
 'NAT1',
 'NAT2',
 'SERPINA3',
 'AAMP',
 'AARS']

In [13]:
# the input is a list of symbol
r = Reactome.run(sybs[:10], organism='Homo sapiens')

In [14]:
# the result
r.table.head()

,name,dbId,found,p-value,fdr,species
0,Acetylation,156582,2,0.000593,0.025498,Homo sapiens
1,Defective SLC6A2 causes orthostatic intoleranc...,5619109,1,0.006619,0.138996,Homo sapiens
2,Amino acid transport across the plasma membrane,352230,2,0.009974,0.139634,Homo sapiens
3,Astrocytic Glutamate-Glutamine Uptake And Meta...,210455,1,0.021899,0.167926,Homo sapiens
4,Neurotransmitter uptake and metabolism In glia...,112313,1,0.021899,0.167926,Homo sapiens


## Gene ontology

In [15]:
# make the association file using the id_mapping function
# detail using will be shown in the utils section
r = IdMapping.convert_to_dict(input_id=c.background, source='ENTREZID', target="GO", species='hsa')

In [16]:
# run go enrichment analysis via goatools 
# the inputs of study, pop, and assoc is list, list,  dict.
# the path is the folder of go obo file
# the path should be a valid filesystem path
path = os.getcwd() + "/go.obo"
rg = GO.run([str(x) for x in c.deg_list], [str(x) for x in c.background], r, obo=path)

load obo file /Users/yangxu/PyPathway/examples/analysis/go.obo
/Users/yangxu/PyPathway/examples/analysis/go.obo: fmt(1.2) rel(2017-12-12) 47,063 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..


15,292 out of 17,216 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
 5,114 out of  5,320 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  15,988 GO terms are associated with 5,114 of 5,320 study items
  21,081 GO terms are associated with 15,292 of 17,216 population items


In [16]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,hit
0,GO:0008150,BP,e,biological_process,4755/5320,14017/17216,5.100000e-78,n.a.,4755,1.070000e-73,1.050000e-73,1.070000e-73,"10, 1000, 10000, 10005, 10006, 10008, 10013, 1..."
1,GO:0009987,BP,e,cellular process,4194/5320,12161/17216,3.650000e-58,n.a.,4194,7.690000e-54,7.500000e-54,7.690000e-54,"10, 1000, 10000, 10005, 10006, 10008, 10013, 1..."
2,GO:0044281,BP,e,small molecule metabolic process,684/5320,1502/17216,1.130000e-35,n.a.,684,2.390000e-31,2.330000e-31,2.390000e-31,"10005, 10057, 10090, 10111, 10135, 10165, 1017..."
3,GO:0006082,BP,e,organic acid metabolic process,409/5320,837/17216,1.120000e-28,n.a.,409,2.370000e-24,2.310000e-24,2.360000e-24,"10005, 10057, 10090, 10170, 10327, 10352, 1036..."
4,GO:0043436,BP,e,oxoacid metabolic process,400/5320,821/17216,1.180000e-27,n.a.,400,2.500000e-23,2.430000e-23,2.490000e-23,"10005, 10057, 10090, 10170, 10327, 10352, 1036..."


In [18]:
# the file input of study, pop and assoc
# this function is the warpper of the Goatools
# Github: https://github.com/tanghaibao/goatools
# cite: Haibao Tang et al. (2015). GOATOOLS: Tools for Gene Ontology. Zenodo. 10.5281/zenodo.31628.
path = "../../tests/assets/data/"
rg = GO.run(path + 'study', path + 'population', path + 'association',
        obo=os.getcwd() + "/go.obo")

obo file not found, start to download
load obo file /Users/yangxu/PyPathway/examples/analysisgo-basic.obo
/Users/yangxu/PyPathway/examples/analysisgo-basic.obo: fmt(1.2) rel(2017-12-24) 47,104 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
goids not found: {'GO:0004221', 'GO:0000300', 'GO:0010149', 'GO:0003702', 'GO:0004428', 'GO:0046909', 'GO:0035300', 'GO:0019204', 'GO:0017163', 'GO:0016563', 'GO:0033903', 'GO:0006800', 'GO:0000059', 'GO:0042156', 'GO:0022625', 'GO:0005624', 'GO:0016564', 'GO:0000739', 'GO:0004437', 'GO:0004086', 'GO:0000299', 'GO:0008159', 'GO:0005625', 'GO:0007108', 'GO:0016944', 'GO:0006496', 'GO:0003840', 'GO:0003711', 'GO:0046853', 'GO:0005678', 'GO:0048196', 'GO:0003701', 'GO:0005792', 'GO:0008943', 'GO:0003715', 'GO:0016566', 'GO:0016585', 'GO:0000072', 'GO:0045750', 'GO:0030528', 'GO:0016251', 'GO:0016986', 'GO:0022627', 'GO:0008471', 'GO:0019575'}


31,855 out of 33,239 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
   269 out of    276 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  787 GO terms are associated with 269 of 276 study items
  6,084 GO terms are associated with 31,847 of 33,239 population items


In [18]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,hit
0,GO:0006464,BP,e,cellular protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0506,0.0493,0.0505,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
1,GO:0036211,BP,e,protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0506,0.0493,0.0505,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
2,GO:0006468,BP,e,protein phosphorylation,22/276,922/33239,0.000011,n.a.,22,0.0661,0.0644,0.0660,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
3,GO:0016310,BP,e,phosphorylation,22/276,996/33239,0.000035,n.a.,22,0.2130,0.2070,0.2130,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
4,GO:0043412,BP,e,macromolecule modification,33/276,1877/33239,0.000057,n.a.,33,0.3520,0.3430,0.3510,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."


In [19]:
# the interactive graph display the significance of the result by color
rg.graph()

In [20]:
# test export
c = EnrichmentExport.export([rg, rg])